## MLOps Remote Model Monitoring Demo
Dec. 2021 / Agent version 8.0.3

<pre>raul.arrabales@datarobot.com</pre>

Adapted from DataRobot Community [6.3.3 version](https://github.com/datarobot-community/custom-models/tree/master/tracking_agents/python)

### Preliminar configuration

In [1]:
# Chech Python version
! python -V

Python 3.7.12


In [2]:
# Mount my google drive so I can access my config files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### MLOps Agent Repo Download


In [3]:
# Clone the MLOps Examples (custom models) repository
# The DataRobot community repo changed name from MLOps Examples to Custom Models
! git clone https://github.com/datarobot-community/mlops-examples

Cloning into 'mlops-examples'...
remote: Enumerating objects: 1407, done.
remote: Counting objects: 100% (591/591), done.
remote: Compressing objects: 100% (306/306), done.
remote: Total 1407 (delta 244), reused 549 (delta 221), pack-reused 816
Receiving objects: 100% (1407/1407), 110.26 MiB | 32.31 MiB/s, done.
Resolving deltas: 100% (612/612), done.


In [4]:
# Install needed packages (requirements.txt from the DR repo)
! pip install -r mlops-examples/tracking_agents/python/requirements.txt

     |████████████████████████████████| 128 kB 8.0 MB/s 
     |████████████████████████████████| 5.9 MB 21.3 MB/s 
     |████████████████████████████████| 147 kB 43.8 MB/s 
     |████████████████████████████████| 5.1 MB 43.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 547 kB 27.0 MB/s 
     |████████████████████████████████| 829 kB 50.7 MB/s 
     |████████████████████████████████| 301 kB 43.9 MB/s 
     |████████████████████████████████| 14.5 MB 22.6 MB/s 
     |████████████████████████████████| 100 kB 7.2 MB/s 
     |████████████████████████████████| 109 kB 66.8 MB/s 
     |████████████████████████████████| 198 kB 66.5 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 227 kB 62.1 MB/s 
     |████████████████████████████████| 509 kB 39.1 MB/s 
     |████████████████████████████████| 269 kB 52.2 MB/s 
     |█

In [45]:
# Install the MLOps Connected Client
! pip install datarobot-mlops-connected-client

     |████████████████████████████████| 1.1 MB 7.8 MB/s 
     |████████████████████████████████| 271 kB 44.8 MB/s 
     |████████████████████████████████| 192 kB 54.7 MB/s 
     |████████████████████████████████| 160 kB 39.9 MB/s 


### Downloading, Configuring and Installing the Agent 


In [1]:
# DataRobot Python API client
import datarobot as dr

import os

In [6]:
# connect to DataRobot platform with python client. 
# using my config file in Google Drive. 
client = dr.Client(config_path='/content/drive/My Drive/Enablement/MLOps/drconfig.yaml')

In [7]:
# Get the tarball from the DR Cloud (HTTP request)
mlops_agents_tb = client.get("mlopsInstaller")

In [8]:
# Write the tarball (HTTP response content) into a local file
with open("mlops-examples/tracking_agents/python/mlops-agent.tar.gz", "wb") as f:
     f.write(mlops_agents_tb.content)

In [9]:
# Uncompress the agent tarball into the root dir
! tar -xf mlops-examples/tracking_agents/python/mlops-agent.tar.gz

In [3]:
# Obtain mlops package folder name and version
with os.popen("ls /content") as pipe:
    for line in pipe:
        if line.startswith('datarobot_mlops_package'):
            mlops_package = line.strip()
            version = line.strip()[-5:]
print(mlops_package)
print(version)

datarobot_mlops_package-8.0.3
8.0.3


In [11]:
# Execute command and install mlops-agent
os.system('pip install /content/{}/lib/datarobot_mlops-{}-py2.py3-none-any.whl'.format(mlops_package, version))

0

In [12]:
# Copy my own MLOps Agent Conf from my Drive. 
# It contains MLOps URL, API Token and Spool Dir. 
os.system('cp /content/drive/My\ Drive/Enablement/MLOps/mlops.agent.conf.yaml /content/{}/conf/mlops.agent.conf.yaml'.format(mlops_package))

0

In [7]:
# Read spool dir from yaml:
import yaml
with open('/content/{}/conf/mlops.agent.conf.yaml'.format(mlops_package)) as conf:
      config_dict = yaml.load(conf, Loader=yaml.BaseLoader)
spool_dir = config_dict.get('channelConfigs')[0].get('details').get('directory')
print(spool_dir)

/content/tmp/ta


In [39]:
# Create the local spool dir
os.system('mkdir -p {}'.format(spool_dir))

0

In [41]:
# Check
! ls -la /content/tmp/ta

total 8
drwxr-xr-x 2 root root 4096 Dec 31 10:48 .
drwxr-xr-x 3 root root 4096 Dec 31 10:48 ..


### Start the agent 

In [42]:
# Start the agent in the local machine
os.system('bash /content/{}/bin/start-agent.sh'.format(mlops_package))
# ! bash /content/datarobot_mlops_package-8.0.3/bin/start-agent.sh

0

In [ ]:
# Shutdown - DON'T RUN THIS CELL, IT'S JUST SHOWING YOU HOW TO SHUTDOWN
# ! bash datarobot_mlops_package-x.x.x/bin/stop-agent.sh

No DataRobot MLOps-Agent is currently running as a service.


In [43]:
# Check that it is running
! bash /content/datarobot_mlops_package-8.0.3/bin/status-agent.sh

DataRobot MLOps-Agent is running as a service.


### Create an MLOps Model Package for a model and deploy it

#### Train a simple RandomForestClassifier model to use for this example

In [4]:
import pandas as pd
import numpy as np
import time
import csv
import pytz
import json
import yaml
import datetime
from sklearn.ensemble import RandomForestClassifier

TRAINING_DATA = '/content/{}/examples/data/mlops-example-surgical-dataset.csv'.format(mlops_package)

df = pd.read_csv(TRAINING_DATA)

columns = list(df.columns)
arr = df.to_numpy()

np.random.shuffle(arr)

split_ratio = 0.8
prediction_threshold = 0.5

train_data_len = int(arr.shape[0] * split_ratio)

train_data = arr[:train_data_len, :-1]
label = arr[:train_data_len, -1]
test_data = arr[train_data_len:, :-1]
test_df = df[train_data_len:]

# train the model
clf = RandomForestClassifier(n_estimators=10, max_depth=2, random_state=0)
clf.fit(train_data, label)

RandomForestClassifier(max_depth=2, n_estimators=10, random_state=0)

#### Create empty deployment in DataRobot MLOps

Using the MLOps client, create a new model package to represent the random forest model we just created.  This includes uploading the traning data and enabling data drift.

In [5]:
from datarobot.mlops.mlops import MLOps
# from datarobot.mlops.common.enums import OutputType
from datarobot.mlops.connected.client import MLOpsClient
from datarobot.mlops.common.exception import DRConnectedException
from datarobot.mlops.constants import Constants

# Read the model configuration info from the example.  This is used to create the model package.
with open('/content/{}/examples/model_config/surgical_binary_classification.json'.format(mlops_package), "r") as f:
    model_info = json.loads(f.read())
model_info

# Read the mlops connection info from the provided example 
with open('/content/{}/conf/mlops.agent.conf.yaml'.format(mlops_package)) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    agent_yaml_dict = yaml.load(file, Loader=yaml.FullLoader)

MLOPS_URL = agent_yaml_dict['mlopsUrl']
API_TOKEN = agent_yaml_dict['apiToken']

# Create connected client
mlops_connected_client = MLOpsClient(MLOPS_URL, API_TOKEN)

# Add training_data to model configuration
print("Uploading training data - {}. This may take some time...".format(TRAINING_DATA))
dataset_id = mlops_connected_client.upload_dataset(TRAINING_DATA)
print("Training dataset uploaded. Catalog ID {}.".format(dataset_id))
model_info["datasets"] = {"trainingDataCatalogId": dataset_id}

# Create the model package
print('Create model package')
model_pkg_id = mlops_connected_client.create_model_package(model_info)
model_pkg = mlops_connected_client.get_model_package(model_pkg_id)
model_id = model_pkg["modelId"]

# Deploy the model package
print('Deploy model package')

# Give the deployment a name:
DEPLOYMENT_NAME="[RAM] SkLearn Remote Model - Binary - " + str(datetime.datetime.now())

deployment_id = mlops_connected_client.deploy_model_package(model_pkg["id"],
                                                            DEPLOYMENT_NAME)

# Enable data drift tracking
print('Enable feature drift')
enable_feature_drift = TRAINING_DATA is not None
mlops_connected_client.update_deployment_settings(deployment_id, target_drift=True,
                                                  feature_drift=enable_feature_drift)
_ = mlops_connected_client.get_deployment_settings(deployment_id)

print("\nDone.")
print("DEPLOYMENT_ID=%s, MODEL_ID=%s" % (deployment_id, model_id))

DEPLOYMENT_ID = deployment_id
MODEL_ID = model_id

Uploading training data - /content/datarobot_mlops_package-8.0.3/examples/data/mlops-example-surgical-dataset.csv. This may take some time...
Training dataset uploaded. Catalog ID 61cee6926efb4af8fffee026.
Create model package
Deploy model package
Enable feature drift

Done.
DEPLOYMENT_ID=61cee6c1693dfb0a76c6f518, MODEL_ID=61cee6c1ccf3698c174d7d69


#### Call the external model's predict fuction and send prediction data to MLOps


You can find Deployment and Model ID under `Deployments` --> `Predictions` --> `Monitoring` Tab. The rest of the code can stay as it is.

In [6]:
# global variables in case runtime is restarted
if 'DEPLOYMENT_ID' not in globals(): 
  DEPLOYMENT_ID = '61cee6c1693dfb0a76c6f518'
if 'MODEL_ID' not in globals():
  MODEL_ID = '61cee6c1ccf3698c174d7d69'

In [9]:
import sys
import time
import random
import pandas as pd

In [10]:
# Binary classifier
CLASS_NAMES = ["1", "0"]

# Spool directory path must match the Monitoring Agent path configured by admin.
SPOOL_DIR = spool_dir

# Actuals dataset
ACTUALS_OUTPUT_FILE = 'actuals.csv'

In [11]:
# MLOps Agent init
mlops = MLOps() \
  .set_deployment_id(DEPLOYMENT_ID) \
  .set_model_id(MODEL_ID) \
  .set_filesystem_spooler(SPOOL_DIR) \
  .init()

In [12]:
# Get predictions
start_time = time.time()
predictions = clf.predict_proba(test_data).tolist()
num_predictions = len(predictions)
end_time = time.time()

# Get assocation id's for the predictions so we can track them with the actuals
def _generate_unique_association_ids(num_samples):
  ts = time.time()
  return ["x_{}_{}".format(ts, i) for i in range(num_samples)]

association_ids = _generate_unique_association_ids(len(test_data))

In [13]:
# MLOPS: report the number of predictions in the request and the execution time.
mlops.report_deployment_stats(num_predictions, end_time - start_time)

True

In [15]:
# MLOPS: report the predictions data: features, predictions, class_names
mlops.report_predictions_data(features_df=test_df, 
                              predictions=predictions, 
                              class_names=CLASS_NAMES,
                              association_ids=association_ids)

True

In [16]:
def write_actuals_file(out_filename, test_data_labels, association_ids):
        """
         Generate a CSV file with the association ids and labels, this example
         uses a dataset that has labels already.
         In a real use case actuals (labels) will show after prediction is done.

        :param out_filename:      name of csv file
        :param test_data_labels:  actual values (labels)
        :param association_ids:   association id list used for predictions
        """
        with open(out_filename, mode="w") as actuals_csv_file:
            writer = csv.writer(actuals_csv_file, delimiter=",")
            writer.writerow(
                [
                    Constants.ACTUALS_ASSOCIATION_ID_KEY,
                    Constants.ACTUALS_VALUE_KEY,
                    Constants.ACTUALS_TIMESTAMP_KEY
                ]
            )
            tz = pytz.timezone("America/Los_Angeles")
            for (association_id, label) in zip(association_ids, test_data_labels):
                actual_timestamp = datetime.datetime.now().replace(tzinfo=tz).isoformat()
                writer.writerow([association_id, "1" if label else "0", actual_timestamp])


In [17]:
target_column_name = columns[len(columns) - 1]
target_values = []
orig_labels = test_df[target_column_name].tolist()

# Write csv file with labels and association Id, when output file is provided
write_actuals_file(ACTUALS_OUTPUT_FILE, orig_labels, association_ids)

print("Wrote actuals file: %s" % ACTUALS_OUTPUT_FILE)

Wrote actuals file: actuals.csv


In [18]:
# MLOPS: release MLOps resources when finished.
mlops.shutdown()

### Upload actuals back to MLOps

In [19]:
def _get_correct_actual_value(deployment_type, value):
    if deployment_type == "Regression":
        return float(value)
    return str(value)

def _get_correct_flag_value(value_str):
    if value_str == "True":
        return True
    return False
    
def upload_actuals():
    print("Connect MLOps client")
    mlops_connected_client = MLOpsClient(MLOPS_URL, API_TOKEN)
    deployment_type = mlops_connected_client.get_deployment_type(DEPLOYMENT_ID)

    actuals = []
    with open(ACTUALS_OUTPUT_FILE, mode="r") as actuals_csv_file:
        reader = csv.DictReader(actuals_csv_file)
        for row in reader:
            actual = {}
            for key, value in row.items():
                if key == Constants.ACTUALS_WAS_ACTED_ON_KEY:
                    value = _get_correct_flag_value(value)
                if key == Constants.ACTUALS_VALUE_KEY:
                    value = _get_correct_actual_value(deployment_type, value)
                actual[key] = value
            actuals.append(actual)

            if len(actuals) == 10000:
                mlops_connected_client.submit_actuals(DEPLOYMENT_ID, actuals)
                actuals = []

    # Submit the actuals
    print("Submit actuals")
    mlops_connected_client.submit_actuals(DEPLOYMENT_ID, actuals)
    
    print("Done.")    

In [20]:
upload_actuals()

Connect MLOps client
Submit actuals
Done.


### Stop the mlops service

In [21]:
! bash /content/datarobot_mlops_package-8.0.3/bin/stop-agent.sh #Change version based on the downloaded file

DataRobot MLOps-Agent shutdown done.
